###1: Setup & Load Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import plotly.express as px
from google.colab import drive
import os

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Define Paths
BASE_PATH = '/content/drive/MyDrive/Statistella_Submission'

Mounted at /content/drive


In [2]:
# Step 1: Define processed data path
PROCESSED_PATH = os.path.join(BASE_PATH, "02_Processed_Data")

# Step 2: Load processed datasets
team_game_stats = pd.read_csv(os.path.join(PROCESSED_PATH, "team_game_stats.csv"))
player_game_stats = pd.read_csv(os.path.join(PROCESSED_PATH, "player_game_stats.csv"))

# Step 3: Parse dates
team_game_stats["GAME_DATE_EST"] = pd.to_datetime(team_game_stats["GAME_DATE_EST"])
player_game_stats["GAME_DATE_EST"] = pd.to_datetime(player_game_stats["GAME_DATE_EST"])

# Step 4: Basic validation
print(team_game_stats.shape)
print(player_game_stats.shape)

/tmp/ipython-input-1231736029.py:6: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  player_game_stats = pd.read_csv(os.path.join(PROCESSED_PATH, "player_game_stats.csv"))


(50532, 22)
(559678, 24)


In [ ]:
INPUT_FILE = os.path.join(BASE_PATH, '02_Processed_Data', 'master_nba_data.csv')
OUTPUT_FILE = os.path.join(BASE_PATH, '02_Processed_Data', 'player_clusters.csv')

# 3. Load Data
print("Loading Master Data...")
df = pd.read_csv(INPUT_FILE)

# 4. Filter for Latest Complete Season
# We analyze the most recent "full" picture of the league
latest_season = df['SEASON'].max()
print(f"Analyzing Season: {latest_season}")

current_players = df[df['SEASON'] == latest_season].copy()
print(f"Total Rows: {current_players.shape[0]}")

Mounted at /content/drive
Loading Master Data...


/tmp/ipython-input-412375781.py:19: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(INPUT_FILE)


Analyzing Season: 2022
Total Rows: 14655


###2: Preprocessing (Aggregating Stats)

In [ ]:
print("Aggregating Player Stats...")

# Group by Player to get seasonal averages
player_stats = current_players.groupby('PLAYER_NAME').agg({
    'PTS': 'mean',
    'REB': 'mean',
    'AST': 'mean',
    'STL': 'mean',
    'BLK': 'mean',
    'TO': 'mean',
    'EFFICIENCY': 'mean',
    'TS_PERCENT': 'mean',
    'GAME_ID': 'count',       # Games Played
    'TEAM_ABBREVIATION': 'first' # Keep Team Tag
}).reset_index()

# Filter: Remove players with < 20 games (removes noise/10-day contracts)
player_stats = player_stats[player_stats['GAME_ID'] >= 20]

print(f"Players Ready for Clustering: {player_stats.shape[0]}")
player_stats.head(3)

Aggregating Player Stats...
Players Ready for Clustering: 414


,PLAYER_NAME,PTS,REB,AST,STL,BLK,TO,EFFICIENCY,TS_PERCENT,GAME_ID,TEAM_ABBREVIATION
1,AJ Green,2.727273,0.818182,0.363636,0.136364,0.000000,0.272727,2.363636,0.277817,22,MIL
2,AJ Griffin,9.277778,2.000000,0.777778,0.888889,0.083333,0.722222,8.055556,0.498471,36,ATL
3,Aaron Gordon,16.031250,6.218750,2.093750,0.906250,0.718750,1.468750,19.093750,0.631702,32,DEN


###3: K-Means Clustering (With "Smart Naming")

In [ ]:
print("Running K-Means Clustering...")

# 1. Select Features
features = ['PTS', 'REB', 'AST', 'STL', 'BLK', 'EFFICIENCY']
X = player_stats[features]

# 2. Scale Data (Standardize range so Points don't dominate Steals)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. Run K-Means (4 Clusters)
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
player_stats['CLUSTER_ID'] = kmeans.fit_predict(X_scaled)

# 4. SMART NAMING (The Fix)
# We calculate the average POINTS for each cluster ID
cluster_means = player_stats.groupby('CLUSTER_ID')['PTS'].mean().sort_values()

# We map the Cluster IDs to a "Rank" (0 = Lowest Scoring, 3 = Highest Scoring)
# This ensures consistency.
rank_map = {old_id: rank for rank, old_id in enumerate(cluster_means.index)}

# Define Names based on Rank
archetype_names = {
    0: 'Role Players / Bench',      # Lowest Avg Points
    1: 'Defensive / Rotation',      # Low-Mid Points
    2: 'Key Starters / Playmakers', # High-Mid Points
    3: 'All-Star Scorers'           # Highest Points
}

# Apply the names
player_stats['ARCHETYPE'] = player_stats['CLUSTER_ID'].map(rank_map).map(archetype_names)

print(" Clustering Complete. Archetypes Assigned.")
print(player_stats[['PLAYER_NAME', 'PTS', 'ARCHETYPE']].sort_values(by='PTS', ascending=False).head(5))

Running K-Means Clustering...
 Clustering Complete. Archetypes Assigned.
                 PLAYER_NAME        PTS         ARCHETYPE
392              Luka Doncic  30.437500  All-Star Scorers
356             Kevin Durant  28.800000  All-Star Scorers
193    Giannis Antetokounmpo  28.800000  All-Star Scorers
276             Jayson Tatum  28.588235  All-Star Scorers
536  Shai Gilgeous-Alexander  27.393939  All-Star Scorers


###4: Validation Plot (Visual Proof)

In [ ]:
# Scatter Plot: Points vs Efficiency colored by Archetype
fig = px.scatter(
    player_stats,
    x='PTS',
    y='EFFICIENCY',
    color='ARCHETYPE',
    hover_name='PLAYER_NAME',
    title=f'<b>AI Analysis: Player Archetypes ({latest_season})</b>',
    template='plotly_dark',
    height=600
)
fig.show()

###5: Save the ML Output

In [ ]:
print(f"Saving ML Results to: {OUTPUT_FILE}")
player_stats.to_csv(OUTPUT_FILE, index=False)
print(" Notebook 2 Finished. Ready for Final Dashboard.")

Saving ML Results to: /content/drive/MyDrive/Statistella_Submission/02_Processed_Data/player_clusters.csv
 Notebook 2 Finished. Ready for Final Dashboard.


## 1. Season-wise Scoring Trends

This visualization shows how average points scored per team per game have changed across NBA seasons.

It helps answer:
- Has scoring increased over time?
- Are modern NBA games more offense-heavy?

This provides a macro-level view of how the league has evolved.

In [19]:
# Step 1: Aggregate average points per season
season_scoring = (
    team_game_stats
    .groupby("SEASON")["PTS"]
    .mean()
    .reset_index()
)

# Step 2: Plot season-wise scoring trend
fig = px.line(
    season_scoring,
    x="SEASON",
    y="PTS",
    markers=True,
    title="Average Points Per Team Per Game by Season"
)

# Step 3: Force all seasons to appear on x-axis
fig.update_xaxes(
    tickmode="array",
    tickvals=season_scoring["SEASON"]
)

fig.show()

## 2. Team-wise Scoring Trends

This chart tracks how individual teams’ scoring output has changed across seasons.

It allows analysts and fans to:
- Compare offensive evolution across franchises
- Identify consistently high-scoring teams
- Spot teams with sharp rises or declines

In [21]:
# Step 1: Compute team-season averages
team_season_scoring = (
    team_game_stats
    .groupby(["SEASON", "NICKNAME"])["PTS"]
    .mean()
    .reset_index()
)

# Step 2: Identify top 5 teams by overall scoring
top_teams = (
    team_season_scoring
    .groupby("NICKNAME")["PTS"]
    .mean()
    .sort_values(ascending=False)
    .head(5)
    .index
)

filtered_team_scoring = team_season_scoring[
    team_season_scoring["NICKNAME"].isin(top_teams)
]

# Step 3: League average
league_avg = (
    team_game_stats
    .groupby("SEASON")["PTS"]
    .mean()
    .reset_index()
)

# Step 4: Plot
fig = px.line(
    filtered_team_scoring,
    x="SEASON",
    y="PTS",
    color="NICKNAME",
    title="Scoring Trends: Top Teams vs League Evolution"
)

fig.add_scatter(
    x=league_avg["SEASON"],
    y=league_avg["PTS"],
    mode="lines",
    name="League Average",
    line=dict(color="black", width=4, dash="dash")
)

# Step 5: Force all seasons to appear on x-axis
all_seasons = sorted(team_season_scoring["SEASON"].unique())

fig.update_xaxes(
    tickmode="array",
    tickvals=all_seasons
)

# Optional readability improvement
fig.update_xaxes(tickangle=-45)

fig.show()


## 3. Home vs Away Performance

This comparison highlights the home-court advantage in the NBA.

It answers:
- Do teams win more often at home?
- How significant is the home advantage historically?

This insight is key to understanding game outcomes and team consistency.


In [23]:
# Step 1: Aggregate win rate by season and home/away
home_away_yearly = (
    team_game_stats
    .groupby(["SEASON", "HOME_AWAY"])["WIN"]
    .mean()
    .reset_index()
)

# Step 2: Plot year-wise home vs away trend
fig = px.line(
    home_away_yearly,
    x="SEASON",
    y="WIN",
    color="HOME_AWAY",
    markers=True,
    title="Year-wise Win Percentage: Home vs Away"
)

# Step 3: Format y-axis as percentage
fig.update_xaxes(
    tickformat=".0%",
    tickmode="array",
    tickvals=all_seasons
)


fig.show()

## 4. Eastern vs Western Conference Dynamics

This visualization compares offensive output between the Eastern and Western Conferences over time.

It helps explore:
- Which conference has been more offense-driven
- Whether dominance shifts across eras

In [24]:
# Step 1: Aggregate scoring by conference and season
conference_scoring = (
    team_game_stats
    .groupby(["SEASON", "CONFERENCE"])["PTS"]
    .mean()
    .reset_index()
)

# Step 2: Plot East vs West trends
fig = px.line(
    conference_scoring,
    x="SEASON",
    y="PTS",
    color="CONFERENCE",
    title="East vs West: Average Points Per Game"
)

fig.update_xaxes(
    tickmode="array",
    tickvals=all_seasons
)


fig.show()

## 5. Most Consistent Teams in the NBA

This visualization highlights the NBA teams with the lowest variability in point differential across seasons, indicating consistent performance levels over time.

Teams such as the **Miami Heat**, **Oklahoma City Thunder**, and **Milwaukee Bucks** appear among the most consistent franchises, suggesting stable team structures and sustained competitiveness.  
Lower point differential variability implies fewer extreme wins or losses, reflecting disciplined play styles and reliable execution.

Overall, this analysis shows that long-term consistency is not limited to championship-winning teams alone but is also evident in franchises that maintain steady performance across different seasons and roster changes.


In [10]:
# Step 1: Compute team-level consistency per season
team_season_consistency = (
    team_game_stats
    .groupby(["SEASON", "NICKNAME"])["POINT_DIFF"]
    .std()
    .reset_index(name="CONSISTENCY")
)

# Step 2: Compute average consistency per team across all seasons

team_avg_consistency = (
    team_season_consistency
    .groupby("NICKNAME")["CONSISTENCY"]
    .mean()
    .reset_index()
    .sort_values("CONSISTENCY")
)

# Step 3: Select top 10 most consistent teams
top_consistent_teams = team_avg_consistency.head(10)

# Step 4: Plot
fig = px.bar(
    top_consistent_teams,
    x="CONSISTENCY",
    y="NICKNAME",
    orientation="h",
    title="Most Consistent Teams in the NBA (Lower Variability in Point Differential)"
)

fig.show()

## 6. League Competitiveness Over Time

This visualization tracks the median team win percentage across NBA seasons, providing a league-wide view of competitive balance.

The median win rate remains close to **50%** throughout most seasons, indicating that the NBA maintains strong parity with no prolonged periods of extreme dominance. Short-term fluctuations reflect transitional seasons, rule changes, or roster shifts, rather than structural imbalance.

Overall, the stability of the median win percentage suggests that competitive balance has been preserved over time, with most teams remaining within a narrow performance range each season.



In [15]:
# Step 1: Compute win percentage per team per season
team_win_pct = (
    team_game_stats
    .groupby(["SEASON", "NICKNAME"])["WIN"]
    .mean()
    .reset_index()
)

# Step 2: Compute league median win percentage per season
league_parity = (
    team_win_pct
    .groupby("SEASON")["WIN"]
    .median()
    .reset_index()
)

# Step 3: Plot with all seasons shown on x-axis
fig = px.line(
    league_parity,
    x="SEASON",
    y="WIN",
    markers=True,
    title="League Competitiveness Over Time (Median Team Win Percentage)"
)

# Force all seasons to appear as x-axis ticks
fig.update_xaxes(
    tickmode="array",
    tickvals=league_parity["SEASON"]
)

fig.update_yaxes(tickformat=".0%")
fig.show()

## 7. Player Archetype Classification

Players are grouped into performance archetypes using clustering on scoring, efficiency, and contribution metrics.

Each archetype represents a distinct on-court role:
- **High-Usage Scorers**: Primary offensive options with heavy scoring responsibility  
- **All-Round Contributors**: Players contributing across scoring, playmaking, and rebounding  
- **Efficient Role Players**: High-impact contributors with limited usage  
- **Low-Impact Bench Players**: Rotational players with lower overall involvement  

This transformation converts complex player-level data into intuitive roles that are easy to analyze and compare.



In [4]:
# Step 1: Aggregate player performance at season level
player_season = (
    player_game_stats
    .groupby(["SEASON", "PLAYER_NAME"])
    .agg({
        "PTS": "mean",
        "REB": "mean",
        "AST": "mean",
        "MINUTES_PLAYED": "mean",
        "PTS_PER_MIN": "mean",
        "EFFICIENCY_SCORE": "mean"
    })
    .reset_index()
)

# Step 2: Keep meaningful contributors only
player_season = player_season[player_season["MINUTES_PLAYED"] >= 18].copy()

# Step 3: Select features for clustering
features = player_season[
    ["PTS", "REB", "AST", "PTS_PER_MIN", "EFFICIENCY_SCORE"]
].copy()

# Step 4: Clean invalid values
features.replace([np.inf, -np.inf], np.nan, inplace=True)
valid_idx = features.dropna().index

features_clean = features.loc[valid_idx]
player_season_clean = player_season.loc[valid_idx]

# Step 5: Scale features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_clean)

# Step 6: Cluster players into archetypes
kmeans = KMeans(n_clusters=4, random_state=42)
player_season_clean["ARCHETYPE"] = kmeans.fit_predict(scaled_features)

# Step 7: Inspect cluster profiles
cluster_profiles = (
    player_season_clean
    .groupby("ARCHETYPE")[["PTS", "REB", "AST", "PTS_PER_MIN", "EFFICIENCY_SCORE", "MINUTES_PLAYED"]]
    .mean()
    .round(2)
)

print(cluster_profiles)

# Step 8: Assign human-readable player type labels
archetype_labels = {
    0: "Low-Impact Bench Players",
    1: "Efficient Role Players",
    2: "High-Usage Scorers",
    3: "All-Round Contributors"
}

player_season_clean["PLAYER_TYPE"] = player_season_clean["ARCHETYPE"].map(archetype_labels)

# Step 9: Visualize player archetypes
fig = px.scatter(
    player_season_clean,
    x="MINUTES_PLAYED",
    y="PTS_PER_MIN",
    color="PLAYER_TYPE",
    size="PTS",
    hover_name="PLAYER_NAME",
    title="NBA Player Archetypes: Role vs Efficiency",
    opacity=0.7
)


# Compute centroids
centroids = (
    player_season_clean
    .groupby("PLAYER_TYPE")[["MINUTES_PLAYED", "PTS_PER_MIN", "PTS"]]
    .mean()
    .reset_index()
)

# Add centroids to plot
fig.add_scatter(
    x=centroids["MINUTES_PLAYED"],
    y=centroids["PTS_PER_MIN"],
    mode="markers+text",
    marker=dict(size=18, color="black", symbol="x"),
    text=centroids["PLAYER_TYPE"],
    textposition="top center",
    name="Archetype Centers"
)

fig.show()


             PTS   REB   AST  PTS_PER_MIN  EFFICIENCY_SCORE  MINUTES_PLAYED
ARCHETYPE                                                                  
0          13.63  3.52  4.09         0.45             20.57           29.95
1           7.83  3.63  1.66         0.33             13.17           22.98
2          13.41  7.76  1.78         0.45             23.10           29.77
3          22.73  6.25  5.09         0.65             33.11           35.11


## 8. Most Consistent High-Scoring Players

This visualization highlights players who consistently score **20 or more points** in a large proportion of their games, capturing sustained high-impact offensive performance rather than occasional scoring bursts.

Players such as **Kevin Durant**, **LeBron James**, and **Luka Dončić** lead the chart, indicating that they not only score at high volumes but do so with remarkable regularity. The gradual decline across the bars shows how rare it is to maintain elite scoring consistency over long periods.

Overall, this analysis emphasizes that consistent high scoring is a defining trait of true offensive superstars, distinguishing them from players who may have high peaks but less reliability game to game.



In [17]:
player_game_stats["TWENTY_PLUS_PTS"] = (player_game_stats["PTS"] >= 20).astype(int)

player_20pt_consistency = (
    player_game_stats
    .groupby("PLAYER_NAME")["TWENTY_PLUS_PTS"]
    .mean()
    .reset_index()
    .sort_values("TWENTY_PLUS_PTS", ascending=False)
    .head(15)
)

fig = px.bar(
    player_20pt_consistency,
    x="PLAYER_NAME",
    y="TWENTY_PLUS_PTS",
    title="Most Consistent High-Scoring Players (20+ Points Rate)"
)

fig.update_yaxes(tickformat=".0%")
fig.show()

## 9. Standout Player Performances

This table highlights the most impactful individual games based on a composite efficiency score.

These performances often define seasons, rivalries, and iconic moments in NBA history.


In [18]:
# Step 1: Select top 15 single-game performances
top_games = (
    player_game_stats
    .sort_values("EFFICIENCY_SCORE", ascending=False)
    .head(15)
    .copy()
)

# Step 2: Create a label combining player and season
top_games["LABEL"] = top_games["PLAYER_NAME"] + " (" + top_games["SEASON"].astype(str) + ")"

# Step 3: Plot ranked efficiency scores
fig = px.bar(
    top_games.sort_values("EFFICIENCY_SCORE"),
    x="EFFICIENCY_SCORE",
    y="LABEL",
    orientation="h",
    title="Top Single-Game Performances in NBA History (by Efficiency Score)",
    text="EFFICIENCY_SCORE"
)

# Step 4: Improve readability
fig.update_traces(texttemplate="%{text:.1f}", textposition="outside")
fig.update_layout(yaxis_title="", xaxis_title="Efficiency Score")

fig.show()